Check outliers

In [0]:
quantiles = {
    c: dict(
        zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in ["LIMIT_BAL", "AGE","PAY_1","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]
}
quantiles

Out[17]: {'LIMIT_BAL': {'q1': 50000.0, 'q3': 240000.0},
 'AGE': {'q1': 28.0, 'q3': 41.0},
 'PAY_1': {'q1': -1.0, 'q3': 0.0},
 'PAY_2': {'q1': -1.0, 'q3': 0.0},
 'PAY_3': {'q1': -1.0, 'q3': 0.0},
 'PAY_4': {'q1': -1.0, 'q3': 0.0},
 'PAY_5': {'q1': -1.0, 'q3': 0.0},
 'PAY_6': {'q1': -1.0, 'q3': 0.0},
 'BILL_AMT1': {'q1': 3491.0, 'q3': 66866.0},
 'BILL_AMT2': {'q1': 2946.0, 'q3': 63774.0},
 'BILL_AMT3': {'q1': 2618.0, 'q3': 60005.0},
 'BILL_AMT4': {'q1': 2313.0, 'q3': 54574.0},
 'BILL_AMT5': {'q1': 1770.0, 'q3': 50290.0},
 'BILL_AMT6': {'q1': 1271.0, 'q3': 49327.0},
 'PAY_AMT1': {'q1': 995.0, 'q3': 5007.0},
 'PAY_AMT2': {'q1': 817.0, 'q3': 5000.0},
 'PAY_AMT3': {'q1': 390.0, 'q3': 4508.0},
 'PAY_AMT4': {'q1': 292.0, 'q3': 4038.0},
 'PAY_AMT5': {'q1': 260.0, 'q3': 4086.0},
 'PAY_AMT6': {'q1': 134.0, 'q3': 4003.0}}

In [0]:
for i in quantiles:
    iqr = quantiles[i]['q3'] - quantiles[i]['q1']
    quantiles[i]['lower_bound'] = quantiles[i]['q1'] - (iqr * 1.5)
    quantiles[i]['upper_bound'] = quantiles[i]['q3'] + (iqr * 1.5)
print(quantiles)

{'LIMIT_BAL': {'q1': 50000.0, 'q3': 240000.0, 'lower_bound': -235000.0, 'upper_bound': 525000.0}, 'AGE': {'q1': 28.0, 'q3': 41.0, 'lower_bound': 8.5, 'upper_bound': 60.5}, 'PAY_1': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_2': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_3': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_4': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_5': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'PAY_6': {'q1': -1.0, 'q3': 0.0, 'lower_bound': -2.5, 'upper_bound': 1.5}, 'BILL_AMT1': {'q1': 3491.0, 'q3': 66866.0, 'lower_bound': -91571.5, 'upper_bound': 161928.5}, 'BILL_AMT2': {'q1': 2946.0, 'q3': 63774.0, 'lower_bound': -88296.0, 'upper_bound': 155016.0}, 'BILL_AMT3': {'q1': 2618.0, 'q3': 60005.0, 'lower_bound': -83462.5, 'upper_bound': 146085.5}, 'BILL_AMT4': {'q1': 2313.0, 'q3': 54574.0, 'lower_bound': -76078.5, 'upper_bound': 132965.5}, 'BILL_AMT5': {'q1': 1770.0, 'q3': 50290.0, 'lower_bound': -71010.0, 'upper_bound': 123070.0}, 'BILL_AMT6': {'q1': 1271.0, 'q3': 49327.0, 'lower_bound': -70813.0, 'upper_bound': 121411.0}, 'PAY_AMT1': {'q1': 995.0, 'q3': 5007.0, 'lower_bound': -5023.0, 'upper_bound': 11025.0}, 'PAY_AMT2': {'q1': 817.0, 'q3': 5000.0, 'lower_bound': -5457.5, 'upper_bound': 11274.5}, 'PAY_AMT3': {'q1': 390.0, 'q3': 4508.0, 'lower_bound': -5787.0, 'upper_bound': 10685.0}, 'PAY_AMT4': {'q1': 292.0, 'q3': 4038.0, 'lower_bound': -5327.0, 'upper_bound': 9657.0}, 'PAY_AMT5': {'q1': 260.0, 'q3': 4086.0, 'lower_bound': -5479.0, 'upper_bound': 9825.0}, 'PAY_AMT6': {'q1': 134.0, 'q3': 4003.0, 'lower_bound': -5669.5, 'upper_bound': 9806.5}}

In [0]:

import pyspark.sql.functions as f
df_clean=df.select(
    "*",
    *[
        f.when(
            f.col(c).between(quantiles[c]['lower_bound'], quantiles[c]['upper_bound']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c in ["LIMIT_BAL", "AGE","PAY_1","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]
    ]
)

In [0]:
from pyspark.sql.functions import col
df_clean=df_clean.withColumn("outliers", col("LIMIT_BAL_out")+col("AGE_out")+col("PAY_1_out")+col("BILL_AMT1_out")+col("PAY_AMT1_out")+col("PAY_2_out")+col("BILL_AMT2_out")+col("PAY_AMT2_out")+col("PAY_3_out")+col("BILL_AMT3_out")+col("PAY_AMT3_out")+col("PAY_4_out")+col("BILL_AMT4_out")+col("PAY_AMT4_out")+col("PAY_5_out")+col("BILL_AMT5_out")+col("PAY_AMT5_out")+col("PAY_6_out")+col("BILL_AMT6_out")+col("PAY_AMT6_out"))

In [0]:
df_clean.select("outliers","ID").groupBy("outliers").agg(count("id")).orderBy('outliers').display()

outliers,count(id)
0,12827
1,5797
2,3040
3,2001
4,1397
5,1044
6,1635
7,454
8,259
9,188
